# Set up

In [ ]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm


import os
import pickle

import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

# data

In [ ]:
X_train = pd.read_csv('Data/Data20260110/X_train_Final_10.csv', header=None, delimiter=',').values
Y_train_pca = pd.read_csv('Data/Data20260110/Y_data_train_pca.csv', header=None, delimiter=',').values

X_test = pd.read_csv('Data/Data20260110/X_test.csv', header=None, delimiter=',').values
Y_test_pca = pd.read_csv('Data/Data20260110/Y_data_test_pca.csv', header=None, delimiter=',').values

X_test_MI_1 = pd.read_csv('LocalDisease/X_MI_1_5.csv', header=None, delimiter=',').values
Y_test_MI_1 = pd.read_csv('LocalDisease/Y_data_MI_1_pca.csv', header=None, delimiter=',').values

X_test_MI_2 = pd.read_csv('LocalDisease/X_MI_2.csv', header=None, delimiter=',').values
Y_test_MI_2 = pd.read_csv('LocalDisease/Y_data_MI_2_pca.csv', header=None, delimiter=',').values

X_test_MI_3 = pd.read_csv('LocalDisease/X_MI_3.csv', header=None, delimiter=',').values
Y_test_MI_3 = pd.read_csv('LocalDisease/Y_data_MI_3_pca.csv', header=None, delimiter=',').values

X_test_HCM = pd.read_csv('LocalDisease/X_HCM.csv', header=None, delimiter=',').values
Y_test_HCM = pd.read_csv('LocalDisease/Y_data_HCM_pca.csv', header=None, delimiter=',').values

train_x = torch.tensor(X_train, dtype=torch.float32)
train_y_21 = torch.tensor(Y_train_pca, dtype=torch.float32)

test_x = torch.tensor(X_test, dtype=torch.float32)
test_y_21 = torch.tensor(Y_test_pca, dtype=torch.float32)

test_x_MI_1 = torch.tensor(X_test_MI_1, dtype=torch.float32)
test_y_MI_1 = torch.tensor(Y_test_MI_1, dtype=torch.float32)

test_x_MI_2 = torch.tensor(X_test_MI_2, dtype=torch.float32)
test_y_MI_2 = torch.tensor(Y_test_MI_2, dtype=torch.float32)

test_x_MI_3 = torch.tensor(X_test_MI_3, dtype=torch.float32)
test_y_MI_3 = torch.tensor(Y_test_MI_3, dtype=torch.float32)

test_x_HCM = torch.tensor(X_test_HCM, dtype=torch.float32)
test_y_HCM = torch.tensor(Y_test_HCM, dtype=torch.float32)



torch.set_default_dtype(torch.float32)

# Emulator

In [ ]:
Device = 'cuda'

In [ ]:
MVGP_models, MVGP_likelihoods = Training.train_MultitaskVGP_minibatch(
    train_x=train_x.to(Device),
    train_y=train_y_21.to(Device),
    covar_type='RQ',
    num_latents=24,
    num_inducing=500,
    lr_hyper=0.01,
    lr_variational=0.1,
    num_iterations=15000,
    patience=10,
    device=Device,
    batch_size=512,
    eval_every=100,
    eval_batch_size=1024
)

In [ ]:
checkpoint = torch.load('multitask_gp_checkpoint_LocalD.pth', map_location=Device)
model_params = checkpoint['model_params']

MVGP_models = GP_models.MultitaskVariationalGP(train_x, train_y_21, 
                                               num_latents=model_params['num_latents'], 
                                               num_inducing=model_params['num_inducing'], 
                                               covar_type=model_params['covar_type']).to(Device)

MVGP_models.load_state_dict(checkpoint['model_state_dict'])

MVGP_likelihoods = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=train_y_21.shape[1]).to(Device)
MVGP_likelihoods.load_state_dict(checkpoint['likelihood_state_dict'])

MVGP_models.eval()
MVGP_likelihoods.eval()


In [ ]:
from sklearn.metrics import r2_score

preds_tmp = Prediction.preds_for_one_model(
    MVGP_models, MVGP_likelihoods, test_x.to(Device)
    ).cpu().detach().numpy()


print(r2_score(preds_tmp.reshape(512, 21), test_y_21.numpy()))
print(np.mean((preds_tmp.reshape(512, 21) - test_y_21.numpy()) ** 2))

In [ ]:
from sklearn.metrics import r2_score

preds_tmp = Prediction.preds_for_one_model(
    MVGP_models, MVGP_likelihoods, test_x_MI_1.to(Device)
    ).cpu().detach().numpy()


print(r2_score(preds_tmp.reshape(100, 21), test_y_MI_1.numpy()))
print(np.mean((preds_tmp.reshape(100, 21) - test_y_MI_1.numpy()) ** 2))

In [ ]:
preds_tmp = Prediction.preds_for_one_model(
    MVGP_models, MVGP_likelihoods, test_x_MI_2.to(Device)
    ).cpu().detach().numpy()


print(r2_score(preds_tmp.reshape(100, 21), test_y_MI_2.numpy()))
print(np.mean((preds_tmp.reshape(100, 21) - test_y_MI_2.numpy()) ** 2))

In [ ]:
preds_tmp = Prediction.preds_for_one_model(
    MVGP_models, MVGP_likelihoods, test_x_MI_3.to(Device)
    ).cpu().detach().numpy()


print(r2_score(preds_tmp.reshape(100, 21), test_y_MI_3.numpy()))
print(np.mean((preds_tmp.reshape(100, 21) - test_y_MI_3.numpy()) ** 2))

In [ ]:
preds_tmp = Prediction.preds_for_one_model(
    MVGP_models, MVGP_likelihoods, test_x_HCM.to(Device)
    ).cpu().detach().numpy()


print(r2_score(preds_tmp.reshape(100, 21), test_y_HCM.numpy()))
print(np.mean((preds_tmp.reshape(100, 21) - test_y_HCM.numpy()) ** 2))

# PE

In [ ]:
row_idx = 0

input_point = test_y_21[row_idx, :]

local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y_21, k=100)

bounds = bound.get_bounds(local_train_x)

In [ ]:
preds_tmp = Prediction.preds_for_one_model(
    MVGP_models, MVGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)
    ).cpu().detach().numpy()

In [ ]:
preds_tmp

In [ ]:
input_point

In [ ]:
test_y_21.numpy().shape

In [ ]:
preds_tmp.reshape(100, 21).shape

In [ ]:
estimated_params_tmp, _ = Estimation.multi_start_estimation(
    MVGP_models, MVGP_likelihoods, row_idx, test_y_21, bounds,
    Estimation.estimate_params_for_one_model_Adam, num_starts=8, num_iterations=2000, lr=0.05,
    patience=10, attraction_threshold=0.1, repulsion_strength=0.1, device=Device, show_progress=True
)

In [ ]:
estimated_params_tmp

In [ ]:
test_x[row_idx, :]